<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/mirna_binding/blob/master/notebook/model_train_lock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General Python Modules

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
from multiprocessing import Pool
from functools import partial
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
#from sklearn.metrics import precision_recall_curve

## Data Preprocessing

### download pre-processed ENCORI dataset

The Encori Dataset was obtained by downloading the whole dataset through the
Encori API at [URL](http://www.sysu.edu.cn/403.html).

The dataset is divided into train and test set:

- test set is represented by samples mapping on chromosome 1.  
- train set is represented by any sample except those mapping on chromosome 1.  

The train set should be made of a total number of 179148 samples;

The below cell downloads the train set from the project folder on Github.

The train set corresponds to ENCORI samples not mapping on chromosome 1.

In [ ]:
!wget https://github.com/ML-Bioinfo-CEITEC/mirna_binding/raw/master/data/datasets/train_set_positives.tar.xz
!tar -xvf train_set_positives.tar.xz

## General notebook functions

set of functions to generate the one hot encoding version of sequences and dot matrix.

### convert input nucleotide sequences to arrays

the code below takes as input a table of three columns:

- genomic binding site ( 50nt length);
- microRNA sequence (20nt length);
- label: class [ positive or negative ];

it outputs a list of arrays as:

- 2d matrix of binding vs microRNA with 2 channels. first channel is watson-crick score, second channel is relative position of microRNA (or zero if not required);
- binding site sequence as tensor of shape 50 x 4, where each channel is a nucleotide;
- microRNA sequence as tensor of shape 20 x 4, where each channel is a nucleotide;
- labels: numpy array [ 0 or 1, as negatie or positive];


In [ ]:
def one_hot_encoding(df, tensor_dim=(50,20,1)):
    """
    fun transform input database to one hot encoding numpy array.
    
    parameters:
    df = Pandas df with col names "binding_sequence", "label", "mirna_binding_sequence"
    tensor_dim= 2d matrix shape
    
    output:
    2d dot matrix, labels as np array
    """

    # reset df indexes (needed for multithreading)
    df.reset_index(inplace=True, drop=True)

    # alphabet for watson-crick interactions.
    alphabet = {"AT": 1., "TA": 1., "GC": 1., "CG": 1.} 

    # labels to one hot encoding
    labels = np.where(df.label == 'positive', 1., 0.)

    # create empty main 2d matrix array
    N = df.shape[0] # number of samples in df
    shape_matrix_2d = (N, *tensor_dim) # 2d matrix shape 
    # initialize dot matrix with zeros
    ohe_matrix_2d = np.zeros(shape_matrix_2d, dtype="float32")

    # compile matrix with watson-crick interactions.
    for index, row in df.iterrows():        
        for bind_index, bind_nt in enumerate(row.binding_sequence.upper()):
        
            for mirna_index, mirna_nt in enumerate(
                row.mirna_binding_sequence.upper()
                ):

                base_pairs = bind_nt + mirna_nt
                ohe_matrix_2d[index, bind_index, mirna_index, 0] = alphabet.get(base_pairs, 0)

    return ohe_matrix_2d, labels

### parallelized conversion of an array/dataframe to 2D matrix

The below function takes as input a Pandas df or numpy array, and split it into  batches for parallelization.

Usage:

`output = multithread(df, one_hot_encoding, aux=False, log=False, n_cores=24)`  
`data = join_cores_results(output, aux=True)`

In [ ]:
def join_cores_results(multithread_output):
    """
    join the output of different core processes 
    
    paramenters:
    multithread_out=

    returns:
    concateneted array_2d_matrix and labels of all jobs
    """
    array_2d_matrix = np.concatenate(
        [ process[0] for process in multithread_output ]
    )
    array_labels = np.concatenate(
    [ process[1] for process in multithread_output ]
    )
    return array_2d_matrix, array_labels

In [ ]:
def multithread(df, func, n_cores=4):
    """
    split input dataset into equal parts and parallelize function

    paramenters:
    df=input Pandas dataframe
    fun=applied function
    n_cores=number of cores (def.4)
    """
    iterable = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    lock_func = partial(func)
    df_update = pool.map(lock_func, iterable)
    pool.close()
    pool.join()
    data = join_cores_results(df_update)
    return data

### subsample positive class from main training set

In [ ]:
def positive_sample_generator(df, size, random_state=None, reduce=True):
    """
    random sampling of N samples from main dataframe.

    paramenters:
    df=Pandas dataframe
    size=number of samples to extract (int.)
    random_state=fix seed (int., def. None)
    reduce=remove subsamples from main df (bool, def. True)

    returns:
    positive samples as Pandas dataframe and original dataframe
    """

    #copy input df
    X = df.copy()
    positive_samples = X.sample(
        n=size, random_state=random_state
        ).copy()

    to_drop = [ name for name in X.columns
               if name not in [
                    'mirna_binding_sequence', 'binding_sequence', 'label']
               ]
    positive_samples.drop(
            to_drop, axis=1, inplace=True
            )
    if reduce:
        df = X.drop(positive_samples.index, axis=0, inplace=False
            ).reset_index(drop=True)
    return positive_samples, df

### shuffle positive to create negative

The function generates the negative class by creating a connection between each

binding site and all mirna (expect the real one). If argument mirna_dict is

provided as dictionary of mirna sequences, this dictionary will be used to

create the negative class. Otherwise, all unique mirna sequences of the input

df will be used to generate samples for the negative class.

In [ ]:
def negative_class_generator(df, mirna_list=None, neg_ratio=None,
                             random_state=None):
    """
    random generation of negative samples.

    paramenters:
    df=Pandas dataframe
    mirna_list=list of unique mirna sequences (list)
    neg_ratio=number of false connections for each positive sample (int.)
    random_state=fix seed (int.)

    returns:
    negative samples as Pandas dataframe
    """
    if not mirna_list:
        # generate mirna db of unique sequences
        mirnadb = pd.DataFrame(
            df.mirna_binding_sequence.unique(), columns=['mirnaid']
        )
    else:
        mirnadb = pd.DataFrame(mirna_list)
        mirnadb.columns = ['mirnaid']
    # add mirna db to each row of df
    connections = mirnadb.assign(key=1).merge(
          df.assign(key=1), on='key'
          ).drop(['key', 'label'],axis=1)

    # # find index of positive connection
    positive_samples_mask = (connections.mirnaid == 
                             connections.mirna_binding_sequence)
    # # drop positive connection to create negative samples
    negative_df = connections[~positive_samples_mask].copy().drop(
      ['mirna_binding_sequence'], axis=1
      ).reset_index(drop=True)
    # # rename cols
    negative_df.columns = ['mirna_binding_sequence', 'binding_sequence']
    # # add negative labels
    negative_df['label'] = 'negative'
    if neg_ratio == None:
        return negative_df
    else:
        neg_samples = int(df.shape[0] * neg_ratio)
        return negative_df.sample(n=neg_samples, random_state=random_state)

### generate dataset for training

In [ ]:
def generate_subsets(
    df=None, size=None, neg_ratio=None, pos_df=None,
    update=False, random_state=None, mirna_list=None
    ):
    """
    complete or partial replacement of training dataset;

    paramenters:
    df=Pandas df with all positive samples (def.None)
    size=number of samples to subset from df (int., def.None)
    neg_ratio=retio of negatives to be generated (int., def.None)
    pos_df=subset of positive samples (Pandas df, def.None)
    update=complete (True) or partial (False) replacement (bool, def.False)
    
    returns:
    pos_df, neg_df, df
    """
    if update:
        pos_df, df = positive_sample_generator(
            df, size, random_state=random_state
            )
    neg_df = negative_class_generator(
        pos_df, neg_ratio=neg_ratio,
        random_state=random_state, mirna_list=mirna_list
        )
    return pos_df, neg_df, df


### create Hand-Picked Mini-Batches

Create minibatches keeping the original positive-negative ratio. It returns a list of minibatches (pos + neg together).

In [ ]:
def make_minibatches(pos_samples, neg_samples, split_batch):
    """
    split positive and negative samples dataframes into minibatches. This
    function keeps the positive vs negative ratio within each batch.

    paramenters:
    pos_samples=Pandas df with all positive samples (def.None)
    neg_samples=Pandas df with all negative samples (def.None)
    split_batch=split dataset into N parts (int.)
    
    returns:
    A list where each element is a minibatch
    """
    
    batches_list = []
    ## split subset_pos.class into minibatches
    ### see numpy doc for more details:
    # https://docs.scipy.org/doc/numpy/reference/generated/numpy.split.html
    batch_pos = np.array_split(pos_samples, split_batch)
    ## split subset_neg into minibatches
    ## of size == SPLIT_BATCH
    batch_neg = np.array_split(neg_samples, split_batch)
    ## zip together pos and neg subsets to create minibatches
    for mini_index, minibatch_pairs in enumerate(zip(batch_pos, batch_neg)):
        if log:
            print(
                    '### minibatch pair id is:',
                    mini_index,
                    'pos-neg shapes are:',
                    minibatch_pairs[0].shape[0],
                    minibatch_pairs[1].shape[0],
                    sep='\t'
                    )
        batch_train = pd.concat(minibatch_pairs)
        # append each minibatch to minibatch list
        batches_list.append(batch_train)
    return batches_list

# Build Model

### load TensorFlow and modules

In [ ]:
from tensorflow import keras as keras 
from tensorflow.keras.layers import (
                                BatchNormalization, LeakyReLU,
                                Input, Dense, Conv2D,
                                MaxPooling2D, Flatten, Dropout)

from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam

### model architecture

In [ ]:
def make_arch_00b():
    """
    build model archietecture as described in the manuscript.

    return a model object
    """
    main_input = Input(shape=(50,20,1),
                       dtype='float32', name='main_input'
                       )

    x = Conv2D(
        filters=32,
        kernel_size=(3, 3),
        padding="same",
        data_format="channels_last",
        name="conv_1")(main_input)    
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_1')(x)
    x = Dropout(rate = 0.25)(x)

    x = Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        data_format="channels_last",
        name="conv_2")(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_2')(x)
    x = Dropout(rate = 0.25)(x)

    x = Conv2D(
        filters=128,
        kernel_size=(3, 3),
        padding="same",
        data_format="channels_last",
        name="conv_3")(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), name='Max_3')(x)
    x = Dropout(rate = 0.25)(x)

    conv_flat = Flatten(name='2d_matrix')(x)

    x = Dense(128)(conv_flat)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate = 0.25)(x)

    x = Dense(64)(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate = 0.25)(x)

    x = Dense(32)(x)
    x = LeakyReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate = 0.25)(x)

    main_output = Dense(1, activation='sigmoid', name='main_output')(x)
    model = Model(inputs=[main_input], outputs=[main_output], name='arch_00b')
    
    return model

In [ ]:
def compile_model():
    K.clear_session()

    model = make_arch_00b()
    opt = Adam(
        learning_rate=1e-3,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name="Adam")

    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        )
    return model

### train and evaluation functions

In [ ]:
def train(
    model, minibatch,
    reset_metrics=True,
    save=False,
    fname=None
    ):
    """
    train model on selected minibatch.
    Doc. at https://keras.io/api/models/model_training_apis/.

    paramenters:
    model=compiled Keras model
    minibatch=set of X_train and labels
    reset_metrics=If True, the metrics returned will be only for this batch.
    If False, the metrics will be statefully accumulated across batches.
    save=save model (bool, Def.False)
    fname=model file name

    returns:
    trained model, batch loss
    """

    X_ohe, y_ohe = minibatch  
    model_loss = model.train_on_batch(
        { "main_input" : X_ohe},
        { "main_output" : y_ohe},
        reset_metrics=reset_metrics
        )

    if save:
        model.save(fname)
    return model, model_loss

In [ ]:
def eval_f1(
    model, minibatch, pos_thr=0.5, save=False, fname=None, test=False
    ):
    """
    generate F1 score for the current minibatch
    
    paramenters:
    model=trained model
    minibatch=tuple of X and labels
    pos_thr=threshold for positive class
    save=save model if above f1 threshold (bool, Def.False)
    fname=model file name

    returns:
    f1 score
    """
    X, y_true = minibatch
    if test:
        y_pred = model.predict({'main_input' : X}, batch_size=32)
    else:
        y_pred = model.predict_on_batch({ "main_input" : X})
    
    y_pred_class = np.where(y_pred > pos_thr, 1, 0)
    f1 = f1_score(y_true, y_pred_class)
    return f1

# Run Pipeline

### load Encori Dataset

Load the train set derived from ENCORI database.

In [ ]:
ENCORI_PATH = './train_set_positives.tsv'

train_df = pd.read_csv(
    ENCORI_PATH, sep='\t',
    usecols=['familyseqRepresentative', 'pos_sequence'])
train_df['label'] = 'positive'
train_df.columns = [
                    'binding_sequence',
                    'mirna_binding_sequence',
                    'label'
                    ]

print(f'trainable encori samples are {train_df.shape}')
train_df.head(10)

### create unique miRNA sequences database

The miRNA sequences database is used to create samples of the negative class.

In [ ]:
mirna_db = train_df.mirna_binding_sequence.unique().tolist()

### standard training pipeline

In [ ]:
# define number of positive samples to use for the model training
POSITIVE_SAMPLES = 1000
# define number of negative samples to generate for each positive sample
NEG_RATIO = 10
# set random state for reproducibility
RANDOM_STATE = 1789
np.random.seed(RANDOM_STATE)
# set working directory (models stored here)
WORK_DIR = "./models/train/standard/20K_10neg/"
# custom model name prefix
STRATEGY = "standard"
MODEL_ID = f'model_{STRATEGY}'
# create WORK_DIR if not exists
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)

#### prepare train dataset

In [ ]:
# generate positive and negative samples
pos_df, neg_df, _ = generate_subsets(
    df=train_df, size=POSITIVE_SAMPLES, neg_ratio=NEG_RATIO, pos_df=None,
    update=True, random_state=RANDOM_STATE, mirna_list=mirna_db)
print(f'positive df size is: {pos_df.shape}\nnegative df size is:{neg_df.shape}')
# concateneta positive and negative samples into a unique dataframe
main_df = pd.concat([pos_df, neg_df])
# convert the unique dataframe to one hot encoding
ohe_data = multithread(
            main_df, one_hot_encoding, n_cores=2
            )

main_df_ohe, label_ohe = ohe_data

#### compile model

In [ ]:
model = compile_model()
model.summary()

#### train model

In [ ]:
model_loss = model.fit(
    main_df_ohe, label_ohe,
    validation_split=0.05, epochs=10,
    batch_size=512
    )


## Iterative Dynamic Training

### Define Iterative Train Paramenters
This section allows the user to define train paramenters:  
- POSITIVE_SAMPLES_SIZE: positive sample size for each iteration (int., def. 1000);
- NEG_RATIO: starting ratio for samples of the negative class (int., def. 1);
- RANDOM_STATE: set random state for reproducibility (int., def. 1789);
- CORES: available cores for train dataset pre-processing to ohe (int., def. 4);
- MINIBATCH_SPLIT: number of bins to divide the train dataset (int. def. 500);
- WORK_DIR: save data to PATH (str., def. cwd);
- STRATEGY: training strategy (str: iter or normal);
- MODEL_ID: arbitrary model name (str, def. model);
- MAX_ITER: set max number of iterations (int., def. 10);
- POS_THR: define threshold to claim a prediction as positive (float, def. 0.5)
- F1_TRIGGER: set threshold above that the negative ratio is increased (float, def. 0.75)




In [25]:
POSITIVE_SAMPLES_SIZE = 1000
NEG_RATIO = 1
RANDOM_STATE = 1789
np.random.seed(RANDOM_STATE)
MINIBATCH_SPLIT = 10
CORES = 2
MAX_ITER = 600
POS_THR = 0.7
F1_TRIGGER = 0.7

# set working directory (models stored here)
WORK_DIR = "./models/train/iterative/"
# custom model name prefix
STRATEGY = "iter"
MODEL_ID = f'model_{STRATEGY}'
# create WORK_DIR if not exists
if not os.path.exists(WORK_DIR):
    os.makedirs(WORK_DIR)

### Create Model

In [ ]:
K.clear_session()

model = make_arch_00b()

opt = Adam(
    learning_rate=1e-3,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam")

model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    )


### Iterative Training

This cell defines the iterative training with incresing negative ratio scheme.

In [ ]:
from collections import defaultdict

# copy original data frame
main_df = train_df.copy()
# generate mirna database
mirna_db = main_df.mirna_binding_sequence.unique().tolist()
# initialize array for f1 and val loss metrics
iter_f1_score = np.zeros( (MAX_ITER, 1) )
iter_val_losses = np.zeros( (MAX_ITER, 1) )
# initialize negative ratio
iter_neg_ratio = 0 + NEG_RATIO
# tot pos samples counter
pos_counter = 0
# tot neg samples counter
neg_counter = 0

for iteration in tqdm(range(START_FROM_ITER, MAX_ITER), desc='iteration'):
    # define iteration's model name
    model_name = f'{WORK_DIR}/{iteration}.iterative.h5'
    # evaluate f1 score
    if (iter_f1_score[iteration - 1 ] > F1_TRIGGER:
        # create train dataset by complete replacement
        pos_df, neg_df, main_df = generate_subsets(
            df=main_df, size=POSITIVE_SAMPLES_SIZE,
            neg_ratio=iter_neg_ratio, pos_df=None,
            update=True, random_state=RANDOM_STATE,
            mirna_list=mirna_db)
        pos_counter += pos_df.shape[0]

    else:
        # create train dataset by partial replacement
        pos_df, neg_df, main_df = generate_subsets(
            df=main_df, size=POSITIVE_SAMPLES_SIZE,
            neg_ratio=iter_neg_ratio, pos_df=pos_df,
            update=False, random_state=RANDOM_STATE,
            mirna_list=mirna_db
            )
    # update counter of used negative samples
    neg_counter += neg_df.shape[0]
    print(
        f'#\tstart training of iteration:\t{iteration}\t' +
        f'negative ratio is:\t{iter_neg_ratio}\t' +
        f'total used pos-neg:\t{pos_counter}\t{neg_counter}\t'
        )
    # prepare minibatches
    minibatches = make_minibatches(
        pos_df, neg_df, MINIBATCH_SPLIT
        )

    # create array where minibatch's losses are stored.
    batch_losses = np.zeros( (MINIBATCH_SPLIT, 1), dtype='float')
    for mini_index in tqdm(range(MINIBATCH_SPLIT), desc='minibatches'):
        minibatch = minibatches[mini_index]
        # minibatches to ohe generator
        ohe_data = multithread(
            minibatch, one_hot_encoding, n_cores=CORES
            )
        if mini_index == MINIBATCH_SPLIT - 1:
            # get val loss
            val_loss = model.test_on_batch(
                { "main_input" : ohe_data[0]},
                { "main_output" : ohe_data[1]}
                )
            iter_val_losses[iteration] = val_loss
            # get val f1 score
            f1 = eval_f1(
                        model, ohe_data,
                        f1_thr=POS_THR, save=True,
                        fname=model_name
                         )
            # add val f1 score
            iter_f1_score[iteration] = f1
            # get minibatches average train loss
            avg_loss = np.average(batch_losses)
            
            print(
                f'train loss:\t{round(avg_loss, 8)}\t' +
                f'val loss:\t{round(val_loss, 8)}\t' + 
                f'val f1:{round(f1, 8)}'
                )

        else:
            # train model
            model, batch_loss = train( model, ohe_data, reset_metrics=True )
            # add loss
            batch_losses[mini_index] = batch_loss